# Manipulating axes

Parallelizing axes across multiple dimensions is a key concept for the distributed Bolt array. Bolt is designed so that you don't need to think about this most of the time, and everything is handled automatically. But it's worth understanding how things are structured under the hood, especially for more advanced usage.

## Which axes are parallelized?

They core idea is to use a collection of key-value pairs where the keys and values separately represent axes of a single array. The keys are tuples that encode the indices of the "parallelized" axes, and the values are (smaller) ndarrays representing the "localized" axes. We can choose which axes to parallelize over during construction, and by convention, the key axes come before the value axes. The default is to parallelize over the first axis

In [1]:
from bolt import ones

In [2]:
a = ones((2, 3, 4), sc)

We can see the overall shape

In [3]:
a.shape

(2, 3, 4)

And we can also inspect the shapes of the keys and the values

In [4]:
a.keys.shape

(2,)

In [5]:
a.values.shape

(3, 4)

Because the key axes come before the value axes, we represent which axes are parallelized with a single number, the `split`, which in this case is 1

In [6]:
a.split

1

For those who know Spark: we are using an RDD where each record is a (tuple, ndarray) pair, where in this case the tuple is length 1, and the arrays have shape `(3,4)`. When it's useful to look at the actual records, we can get them with the `tordd` method.

In [7]:
a.tordd().keys().collect()

[(0,), (1,)]

In [8]:
a.tordd().values().first()

array([[ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.]])

If we instead parallelize over the first **two** axes

In [9]:
a = ones((2, 3, 4), sc, axis=(0, 1))

The overall shape is the same, but the shapes of the keys and values have changed

In [10]:
a.shape

(2, 3, 4)

In [11]:
a.keys.shape

(2, 3)

In [12]:
a.values.shape

(4,)

Again, for those who know Spark, we can look at the actual records. We should now have length 2 tuples as the keys, and arrays of shape `(4,)` as the values.

In [13]:
a.tordd().keys().collect()

[(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2)]

In [14]:
a.tordd().values().first()

array([ 1.,  1.,  1.,  1.])

## Transposing

A key operation on distributed arrays is transposing, which lets us apply parallelized operations over different axes

Due to the structure described above, transposes that involve either keys or values are simple because they just involve maps over the keys or the values. That's how the following operations are implemented (though this is handled for you under the hood)

In [15]:
a = ones((2, 3, 4, 5), sc, axis=(0, 1))

In [16]:
a.transpose(0, 1, 3, 2).shape

(2, 3, 5, 4)

In [17]:
a.transpose(1, 0, 2, 3).shape

(3, 2, 4, 5)

The more complex case is when the axes change between keys and values. This is handled through an efficient `swap` method that we developed. We chunk the values into subarrays of moderate size, group them, and reorganize them. The `swap` operation is more general than just a transpose, but it's what we use to perform transposes. In terms of performance, we've used this to efficiently transpose very large (TB+) arrays in minutes on moderately sized clusters.

The transpose itself is lazy, and shapes are computed automatically, so nothing happens until you request an output (usually by calling `sum` or `toarray`). Note the small difference in execution time between the following two operations; only the latter requires execution.

In [18]:
%%time
a.transpose(3, 2, 1, 0).shape

CPU times: user 13.3 ms, sys: 8.32 ms, total: 21.7 ms
Wall time: 53.1 ms


(5, 4, 3, 2)

In [19]:
%%time
a.transpose(3, 2, 1, 0).toarray().shape

CPU times: user 51.3 ms, sys: 10.8 ms, total: 62.1 ms
Wall time: 767 ms


(5, 4, 3, 2)

## Reshaping

Another key operation is reshaping. Like transpose, many reshapings can be performed independently on either the key or the value axes, and this is handled automatically given an arbtirary reshape.

In [20]:
a.shape

(2, 3, 4, 5)

In [21]:
a.reshape(3, 2, 4, 5).shape

(3, 2, 4, 5)

In [22]:
a.reshape(2, 3, 5, 4).shape

(2, 3, 5, 4)

We have not yet solved the more general case (where the reshaping spans multiple axes).

## Mapping over axes

As shown in the basic usage tutorial, we can apply a function over our distributed arrays in parallel using `map`. We can apply our function over any axes, but depending on the axes we choose, the implementation differs. If we map over the axes represented in the values, then it is simply a single `map` operation. If we map over another set of axes, it will require a `swap` to move those axes

For example, this does not incur a `swap`

In [23]:
a = ones((2, 3, 4, 5), sc, axis=(0, 1))

In [24]:
a.map(lambda x: x * 2, axis=(0, 1)).toarray().shape

(2, 3, 4, 5)

but this does

In [25]:
a.map(lambda x: x * 2, axis=(0,)).toarray().shape

(2, 3, 4, 5)

Understanding and controlling which axes are parallelized will defintiely affect the performance of distributed operations like `map`!